In [1]:
import os
import torch
import math

import pandas as pd

from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter
from utils import train_loop, val_loop
from model import MyDataset, CryptoTransformer, MSEPlusRank

In [2]:
# hyperparameters
BATCH_SIZE = 64
NUM_FEATURES = 68
MODEL_DIM = 64
FFN_DIM = 128
DROPOUT = 0.3
NUM_HEADS = 2
NUM_LAYERS = 2
MAX_EPOCH = 10
experiment_name = 'model_v1.1'

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [3]:
train_length = 24236806
num_splits = 10
split_length = math.floor(train_length/num_splits)

processed_val = pd.read_csv('./processed_data/processed_val.gz')
val_dataset = MyDataset(processed_val)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [4]:
net = CryptoTransformer(model_dim=MODEL_DIM, num_heads=NUM_HEADS, num_layers=NUM_LAYERS, dropout=DROPOUT, ffn_dim=FFN_DIM, num_features=NUM_FEATURES).to(device)
loss_fn = MSEPlusRank(lamda=1)
optimizer = torch.optim.AdamW(net.parameters())
total_num = sum(p.numel() for p in net.parameters())
trainable_num = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"Total number of parameters: {total_num/1e3:.0f}K, number of trainable parameters: {trainable_num/1e3:.0f}K")

Total number of parameters: 105K, number of trainable parameters: 105K


In [5]:
if not os.path.exists(f'./checkpoint/{experiment_name}'):
    os.mkdir(f'./checkpoint/{experiment_name}')
if not os.path.exists(f'./logs/{experiment_name}'):
    os.mkdir(f'./logs/{experiment_name}')

In [6]:
min_val_loss = float('inf')
best_epoch = 1
best_step = 1
step = 0

tb = SummaryWriter(log_dir=f'./logs/{experiment_name}/')
for t in range(MAX_EPOCH):
    for i in range(num_splits):
        print(f"Epoch {t+1}, Train Split: {i+1}\n--------------------------")
        processed_train = pd.read_csv('./processed_data/processed_train.gz', skiprows=range(1,split_length*2), nrows=split_length)        
        train_dataset = MyDataset(processed_train)
        train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        train_loss = train_loop(train_dataloader, net, loss_fn, optimizer, device)

        val_loss = val_loop(val_dataloader, net, loss_fn, device)
        tb.add_scalar("Train Loss", train_loss, t*num_splits+i+1)
        tb.add_scalar("Val Loss", val_loss, t*num_splits+i+1)
    
        torch.save(net, f"./checkpoint/{experiment_name}/epoch_{t}_step_{i+1}.pt")
        if val_loss < min_val_loss:
            best_epoch = t+1
            best_step = i+1
            min_val_loss = val_loss
print(f"best epoch: {best_epoch}, best step: {best_step}, minimun validations loss: {min_val_loss:.2e}")

Epoch 1, Train Split: 1
--------------------------


  0%|          | 0/1266 [00:00<?, ?it/s]/Users/luoxinyang/opt/anaconda3/envs/gresearch/lib/python3.9/site-packages/torch/nn/modules/transformer.py:380: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1699448801785/work/aten/src/ATen/NestedTensorImpl.cpp:180.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)
100%|██████████| 1266/1266 [00:29<00:00, 42.45it/s, val loss=0.011]  


Epoch 1, Train Split: 2
--------------------------


100%|██████████| 1266/1266 [00:36<00:00, 34.72it/s, val loss=0.00726]


Epoch 1, Train Split: 3
--------------------------


 88%|████████▊ | 2818/3210 [01:42<00:14, 27.06it/s, train loss=3.78e-5]